In [1]:
import pandas as pd
import numpy as np
from itertools import product
# import soccerdata as sd

In [2]:
def collapse_cols(df):
    df.columns = ['_'.join(col).strip('_').strip() for col in df.columns.values]
    return df

def count_pos(x, pos):
    if pos in x:
        return 1
    else:
        return 0
    
def add_pos(df):
    df['AM'] = df['Pos'].apply(count_pos, pos = 'AM')
    df['CM'] = df['Pos'].apply(count_pos, pos = 'CM')
    df['DM'] = df['Pos'].apply(count_pos, pos = 'DM')
    df['RM'] = df['Pos'].apply(count_pos, pos = 'RM')
    df['LM'] = df['Pos'].apply(count_pos, pos = 'LM')
    df['CB'] = df['Pos'].apply(count_pos, pos = 'CB')
    df['LB'] = df['Pos'].apply(count_pos, pos = 'LB')
    df['RB'] = df['Pos'].apply(count_pos, pos = 'RB')
    df['RW'] = df['Pos'].apply(count_pos, pos = 'RW')
    df['LW'] = df['Pos'].apply(count_pos, pos = 'LW')
    df['FW'] = df['Pos'].apply(count_pos, pos = 'FW')
    df['WB'] = df['Pos'].apply(count_pos, pos = 'WB')
    df['GK'] = df['Pos'].apply(count_pos, pos = 'GK')
    return df

In [3]:
schedule = pd.read_parquet('../input/dataset-for-forecasting/schedule.pq')

In [4]:
schedule = schedule.reset_index()
schedule = schedule[~schedule.game_id.isna()]
gamedict = dict(zip(schedule.game_id, schedule.game))
seasondict = dict(zip(schedule.game_id, schedule.season))

In [5]:
schedule.season.value_counts()

1718    380
1819    380
2021    380
2122    380
Name: season, dtype: int64

In [6]:
schedule.game_id.value_counts().sort_values()

e3c3ddf0    1
7c834541    1
e782371e    1
60f6cc1d    1
684f704a    1
           ..
7b4b63d0    1
82aa061a    1
5fb752fa    1
076ca089    1
771ecae9    1
Name: game_id, Length: 1520, dtype: int64

In [7]:
sum1718 = pd.read_pickle('../input/fbref2/data/data/fbrefplayerdata/1718/player_sums.pk').reset_index()

In [8]:
dflist = [
    '../input/fbref2/data/data/fbrefplayerdata/1819/player_summary1819.pq',
    '../input/fbref2/data/data/fbrefplayerdata/1920/player_summary1920.pq',
    '../input/fbref2/data/data/fbrefplayerdata/2021/player_summary2021.pq',
    '../input/fbref2/data/data/fbrefplayerdata/2122/player_summary2021.pq',
]

In [9]:
dfs = []
dfs.append(sum1718)
for df in dflist:
    df = pd.read_parquet(df).reset_index()
    dfs.append(df)

In [10]:
dfs[0].columns

MultiIndex([(     'league',        ''),
            (     'season',        ''),
            (       'game',        ''),
            (       'team',        ''),
            (     'player',        ''),
            (    'Carries', 'Carries'),
            (    'Carries',    'Prog'),
            (   'Dribbles',     'Att'),
            (   'Dribbles',    'Succ'),
            (   'Expected',    'npxG'),
            (   'Expected',      'xA'),
            (   'Expected',      'xG'),
            (     'Passes',     'Att'),
            (     'Passes',     'Cmp'),
            (     'Passes',    'Cmp%'),
            (     'Passes',    'Prog'),
            ('Performance',     'Ast'),
            ('Performance',  'Blocks'),
            ('Performance',    'CrdR'),
            ('Performance',    'CrdY'),
            ('Performance',     'Gls'),
            ('Performance',     'Int'),
            ('Performance',      'PK'),
            ('Performance',   'PKatt'),
            ('Performance',   'Press'),


In [11]:
for df in dflist:
    print(pd.read_parquet(df).reset_index().season.unique())

['1819' nan]
['1920' nan]
['2021' nan]
['2122' nan]


In [12]:
df = pd.concat(dfs)
df.season

0        1718
1        1718
2        1718
3        1718
4        1718
         ... 
10481     NaN
10482     NaN
10483     NaN
10484     NaN
10485     NaN
Name: season, Length: 52425, dtype: object

In [13]:
df[df.game_id.isna()]

Empty DataFrame
Columns: [(league, ), (season, ), (game, ), (team, ), (player, ), (Carries, Carries), (Carries, Prog), (Dribbles, Att), (Dribbles, Succ), (Expected, npxG), (Expected, xA), (Expected, xG), (Passes, Att), (Passes, Cmp), (Passes, Cmp%), (Passes, Prog), (Performance, Ast), (Performance, Blocks), (Performance, CrdR), (Performance, CrdY), (Performance, Gls), (Performance, Int), (Performance, PK), (Performance, PKatt), (Performance, Press), (Performance, Sh), (Performance, SoT), (Performance, Tkl), (Performance, Touches), (SCA, GCA), (SCA, SCA), (#, ), (Nation, ), (Pos, ), (Age, ), (Min, ), (game_id, ), (week, )]
Index: []

[0 rows x 38 columns]

In [14]:
df['season'] = df['game_id'].map(seasondict)

In [15]:
df['game_id'].map(seasondict)

0        1718
1        1718
2        1718
3        1718
4        1718
         ... 
10481    2122
10482    2122
10483    2122
10484    2122
10485    2122
Name: game_id, Length: 52425, dtype: object

In [16]:
df['game'] = df['game_id'].map(gamedict)

In [17]:
df.season.unique()

array(['1718', '1819', nan, '2021', '2122'], dtype=object)

In [18]:
df = df.drop('league', axis=1)

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [19]:
df = collapse_cols(df)
# df = add_pos(df)
# df['midfielder'] = (df.CM + df.AM + df.DM + df.RM + df.LM).apply(lambda x: x > 0)
# df['defender'] = (df.CB + df.LB + df.RB + df.WB).apply(lambda x: x > 0)
# df['attacker'] = (df.AM + df.LW + df.RW + df.FW).apply(lambda x: x > 0)
# midfielders = df[df.midfielder].groupby(['team', 'week'])[interest_cols].sum().add_suffix('_mid')
# attackers = df[df.attacker].groupby(['team', 'week'])[interest_cols].sum().add_suffix('_att')
# defenders = df[df.defender].groupby(['team', 'week'])[interest_cols].sum().add_suffix('_def')
# keepers =df[df.GK.astype(bool)].groupby(['team', 'week'])[['Passes_Att', 'Passes_Cmp', 'Passes_Prog', 'Tackles', 'Blocks']].sum().add_suffix('_gk')

In [20]:
played = schedule.copy()
played['home_score'] = played.score.apply(lambda x: x.split('–')[0])
played['away_score'] = played.score.apply(lambda x: x.split('–')[-1])
teams = played.home_team.unique()
weeks = played.week.unique()
seasons = played.season.unique()
index = pd.MultiIndex.from_product([teams, weeks, seasons])
teamdf = pd.DataFrame(index = index, columns = ["Home", "Away", "GF", "GA", "W", "D", "L", "Opponent"])
teamdf['D'] = teamdf['W'] = teamdf['L'] = 0
teamdf["Opponent"] = ""
for i, game in played.iterrows():
    hometeam = game['home_team']
    awayteam = game['away_team']
    week = game['week']
    season = game['season']
    teamdf.loc[(hometeam, week, season), "GF"] = game['home_score']
    teamdf.loc[(awayteam, week, season), "GF"] = game['away_score']
    teamdf.loc[(hometeam, week, season), "GA"] = game['away_score']
    teamdf.loc[(awayteam, week, season), "GA"] = game['home_score']
    
    teamdf.loc[(hometeam, week, season), "Home"] = 1
    teamdf.loc[(hometeam, week, season), "Away"] = 0
    
    teamdf.loc[(awayteam, week, season), "Home"] = 0
    teamdf.loc[(awayteam, week, season), "Away"] = 1
    # Set Opponents
    teamdf.loc[(hometeam, week, season), "Opponent"] = awayteam
    teamdf.loc[(awayteam, week, season), "Opponent"] = hometeam
    if game['home_score'] == game['away_score']:
#         print(f'draw, {hometeam}, {awayteam}')
        teamdf.loc[(hometeam, week, season), 'D'] = 1
        teamdf.loc[(awayteam, week, season), 'D'] = 1
    elif game['home_score'] > game['away_score']:
#         print(f'homewin, {hometeam}, {awayteam}')
        teamdf.loc[(hometeam, week, season), 'W'] = 1
        teamdf.loc[(awayteam, week, season), 'L'] = 1
    else:
#         print(f'awawin, {hometeam}, {awayteam}')
        teamdf.loc[(hometeam, week, season), 'L'] = 1
        teamdf.loc[(awayteam, week, season), 'W'] = 1
        

played['home_score'] = played['home_score'].apply(pd.to_numeric)
played['away_score'] = played['away_score'].apply(pd.to_numeric)


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2975: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures, cell_id


In [21]:
teamdf = teamdf.reset_index().rename(columns = {'level_0': 'team', 'level_1': 'week', 'level_2': 'season'})

In [22]:
teamdf.sort_values(by='team')

,team,week,season,Home,Away,GF,GA,W,D,L,Opponent
0,Arsenal,1.0,1718,1,0,4,3,1,0,0,Leicester City
97,Arsenal,25.0,1819,0,1,1,3,0,0,1,Manchester City
98,Arsenal,25.0,2021,1,0,0,1,0,0,1,Manchester City
99,Arsenal,25.0,2122,0,1,4,2,1,0,0,Chelsea
100,Arsenal,26.0,1718,1,0,5,1,1,0,0,Everton
...,...,...,...,...,...,...,...,...,...,...,...
3291,Wolves,25.0,2122,0,1,2,0,1,0,0,Tottenham
3290,Wolves,25.0,2021,1,0,1,0,1,0,0,Leeds United
3289,Wolves,25.0,1819,0,1,3,1,1,0,0,Everton
3287,Wolves,24.0,2122,1,0,0,1,0,0,1,Arsenal


In [23]:
 def leven(x, y):
    """
    String Levenshtein distance (edit distance)
    From https://devrescue.com/levenshtein-distance-in-python/
    """
    n = len(x)
    m = len(y)
    
    x_l = x.lower()
    y_l = y.lower()

    A = [[i + j for j in range(m + 1)] for i in range(n + 1)]

    for i in range(n):
        for j in range(m):
            A[i + 1][j + 1] = min(A[i][j + 1] + 1,              # insert
                                  A[i + 1][j] + 1,              # delete
                                  A[i][j] + int(x_l[i] != y_l[j]))  # replace

    return A[n][m] / max(len(x), len(y))

In [24]:
teams_tdf = teamdf.team.unique()
teams_df = df.team.unique()

In [25]:
team_name_leven = {}

In [26]:
for team_a in teams_tdf:
    min_distance = -1
    shortest = ""
    for team_b in teams_df:
        dist = leven(team_a, team_b)
        if min_distance < 0:
            min_distance = dist
            shortest = team_b
        if (dist < min_distance):
#             print(f"distance for {team_a} to {team_b} is {dist}")
            shortest = team_b
            min_distance = dist
    team_name_leven[team_a] = shortest
    print(f"matched {team_a} to {team_name_leven[team_a]}")

matched Arsenal to Arsenal
matched Brighton to Everton
matched Chelsea to Chelsea
matched Crystal Palace to Crystal Palace
matched Everton to Everton
matched Southampton to Southampton
matched Watford to Watford
matched West Brom to West Bromwich Albion
matched Manchester Utd to Manchester United
matched Newcastle Utd to Newcastle United
matched Bournemouth to Bournemouth
matched Burnley to Burnley
matched Leicester City to Leicester City
matched Liverpool to Liverpool
matched Stoke City to Stoke City
matched Swansea City to Swansea City
matched Huddersfield to Huddersfield Town
matched Tottenham to Tottenham Hotspur
matched Manchester City to Manchester City
matched West Ham to West Ham United
matched Fulham to Fulham
matched Wolves to Chelsea
matched Cardiff City to Cardiff City
matched Sheffield Utd to Sheffield United
matched Leeds United to Leeds United
matched Aston Villa to Aston Villa
matched Brentford to Brentford
matched Norwich City to Norwich City


In [27]:
team_name_leven['Brighton'] = 'Brighton & Hove Albion'
team_name_leven['Wolves'] = 'Wolverhampton Wanderers'

In [28]:
teamdf.team.unique()

array(['Arsenal', 'Brighton', 'Chelsea', 'Crystal Palace', 'Everton',
       'Southampton', 'Watford', 'West Brom', 'Manchester Utd',
       'Newcastle Utd', 'Bournemouth', 'Burnley', 'Leicester City',
       'Liverpool', 'Stoke City', 'Swansea City', 'Huddersfield',
       'Tottenham', 'Manchester City', 'West Ham', 'Fulham', 'Wolves',
       'Cardiff City', 'Sheffield Utd', 'Leeds United', 'Aston Villa',
       'Brentford', 'Norwich City'], dtype=object)

In [29]:
df.team.unique()

array(['Leicester City', 'Arsenal', 'Manchester City',
       'Brighton & Hove Albion', 'Burnley', 'Chelsea',
       'Huddersfield Town', 'Crystal Palace', 'Stoke City', 'Everton',
       'Swansea City', 'Southampton', 'Liverpool', 'Watford',
       'Bournemouth', 'West Bromwich Albion', 'West Ham United',
       'Manchester United', 'Tottenham Hotspur', 'Newcastle United',
       'Cardiff City', 'Fulham', 'Wolverhampton Wanderers',
       'Norwich City', 'Sheffield United', 'Aston Villa', 'Leeds United',
       'Brentford'], dtype=object)

In [30]:
team_map = {
    'Arsenal':'Arsenal', 'Brighton':'Brighton', 'Chelsea':'Chelsea', 'Crystal Palace':'Crystal Palace', 'Everton':'Everton',
       'Southampton':'Southampton', 'Watford':'Watford', 'West Brom':'West Bromwich Albion', 'Manchester Utd':'Manchester United',
       'Newcastle Utd':'Newcastle United', 'Bournemouth':'Bournemouth', 'Burnley':'Burnley', 'Leicester City':'Leicester City',
       'Liverpool':'Liverpool', 'Stoke City':'Stoke City', 'Swansea City':'Swansea City', 'Huddersfield':'Huddersfield Town',
       'Tottenham':'Tottenham Hotspur', 'Manchester City':'Manchester City', 'West Ham':'West Ham United', 'Fulham':'Fulham', 'Wolves':'Wolverhampton Wanderers',
       'Cardiff City':'Cardiff City', 'Sheffield Utd':'Sheffield United', 'Leeds United':'Leeds United', 'Aston Villa':'Aston Villa',
       'Brentford':'Brentford', 'Norwich City':'Norwich City'
}

In [31]:
teamdf.team = teamdf.team.map(team_name_leven)
teamdf.Opponent = teamdf.Opponent.map(team_name_leven)

In [32]:
df.drop(['#',
       'Age',
       'Min'], 1).groupby(['team',
                           'week',
                           'season']).mean().add_suffix("_mean").reset_index().season.unique()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


array(['1718', '1819', '2021', '2122'], dtype=object)

In [33]:
game_ids = df.drop(['#','Age','Min'], 1).groupby(['team','week','season'])['game_id'].first()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [34]:
df_to_merge = df.drop(['#','Age','Min'], 1).groupby(['team','week','season']).mean().add_suffix("_mean")
df_to_merge = pd.concat([df_to_merge, game_ids], axis=1)
df_to_merge = df_to_merge.reset_index()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [35]:
teamdf = teamdf.merge(df_to_merge,on=['team','week','season'], how = 'left' )
teamdf

,team,week,season,Home,Away,GF,GA,W,D,L,...,Performance_PK_mean,Performance_PKatt_mean,Performance_Press_mean,Performance_Sh_mean,Performance_SoT_mean,Performance_Tkl_mean,Performance_Touches_mean,SCA_GCA_mean,SCA_SCA_mean,game_id
0,Arsenal,1.0,1718,1,0,4,3,1,0,0,...,0.0,0.0,6.571429,1.928571,0.714286,1.571429,57.714286,0.500000,3.571429,e3c3ddf0
1,Arsenal,1.0,1819,1,0,0,2,0,0,1,...,0.0,0.0,15.642857,0.642857,0.214286,1.571429,39.357143,0.000000,1.142857,478e9dab
2,Arsenal,1.0,2021,0,1,3,0,1,0,0,...,0.0,0.0,9.142857,0.928571,0.357143,0.857143,53.428571,0.285714,1.571429,bf52349b
3,Arsenal,1.0,2122,0,1,0,2,0,0,1,...,0.0,0.0,7.642857,1.571429,0.285714,0.785714,52.500000,0.000000,3.000000,3adf2aa7
4,Arsenal,2.0,1718,0,1,0,1,0,0,1,...,0.0,0.0,7.642857,1.357143,0.500000,1.071429,63.428571,0.000000,1.928571,37f97732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4251,Norwich City,37.0,2122,0,1,1,1,0,1,0,...,0.0,0.0,12.857143,0.785714,0.142857,1.714286,33.214286,0.142857,1.357143,7f0d5ca8
4252,Norwich City,38.0,1718,NaN,NaN,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4253,Norwich City,38.0,1819,NaN,NaN,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4254,Norwich City,38.0,2021,NaN,NaN,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df_to_merge = df.drop(['#','Age','Min'], 1).groupby(['team','week','season']).sum().add_suffix("_sum").reset_index()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [37]:
exp_cols = [col for col in df_to_merge.columns if 'expected' in col.lower()]
df_to_merge[exp_cols]

,Expected_npxG_sum,Expected_xA_sum,Expected_xG_sum
0,2.3,2.2,2.3
1,0.6,0.6,0.6
2,1.9,1.0,1.9
3,1.4,1.4,1.4
4,1.5,1.5,1.5
...,...,...,...
3035,0.4,0.2,0.4
3036,0.9,0.6,0.9
3037,0.6,0.6,0.6
3038,0.7,0.4,0.7


In [38]:
teamdf = teamdf.merge(df_to_merge,on=['team','week', 'season'], how = 'left')
teamdf

,team,week,season,Home,Away,GF,GA,W,D,L,...,Performance_Int_sum,Performance_PK_sum,Performance_PKatt_sum,Performance_Press_sum,Performance_Sh_sum,Performance_SoT_sum,Performance_Tkl_sum,Performance_Touches_sum,SCA_GCA_sum,SCA_SCA_sum
0,Arsenal,1.0,1718,1,0,4,3,1,0,0,...,20.0,0.0,0.0,92.0,27.0,10.0,22.0,808.0,7.0,50.0
1,Arsenal,1.0,1819,1,0,0,2,0,0,1,...,12.0,0.0,0.0,219.0,9.0,3.0,22.0,551.0,0.0,16.0
2,Arsenal,1.0,2021,0,1,3,0,1,0,0,...,5.0,0.0,0.0,128.0,13.0,5.0,12.0,748.0,4.0,22.0
3,Arsenal,1.0,2122,0,1,0,2,0,0,1,...,19.0,0.0,0.0,107.0,22.0,4.0,11.0,735.0,0.0,42.0
4,Arsenal,2.0,1718,0,1,0,1,0,0,1,...,12.0,0.0,0.0,107.0,19.0,7.0,15.0,888.0,0.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4251,Norwich City,37.0,2122,0,1,1,1,0,1,0,...,22.0,0.0,0.0,180.0,11.0,2.0,24.0,465.0,2.0,19.0
4252,Norwich City,38.0,1718,NaN,NaN,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4253,Norwich City,38.0,1819,NaN,NaN,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4254,Norwich City,38.0,2021,NaN,NaN,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
exp_cols = [col for col in teamdf.columns if 'expected' in col.lower()]
teamdf[exp_cols]

,Expected_npxG_mean,Expected_xA_mean,Expected_xG_mean,Expected_npxG_sum,Expected_xA_sum,Expected_xG_sum
0,0.164286,0.157143,0.164286,2.3,2.2,2.3
1,0.042857,0.042857,0.042857,0.6,0.6,0.6
2,0.135714,0.071429,0.135714,1.9,1.0,1.9
3,0.100000,0.100000,0.100000,1.4,1.4,1.4
4,0.107143,0.107143,0.107143,1.5,1.5,1.5
...,...,...,...,...,...,...
4251,0.092857,0.078571,0.092857,1.3,1.1,1.3
4252,NaN,NaN,NaN,NaN,NaN,NaN
4253,NaN,NaN,NaN,NaN,NaN,NaN
4254,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
teamdf.team.unique()

array(['Arsenal', 'Brighton & Hove Albion', 'Chelsea', 'Crystal Palace',
       'Everton', 'Southampton', 'Watford', 'West Bromwich Albion',
       'Manchester United', 'Newcastle United', 'Bournemouth', 'Burnley',
       'Leicester City', 'Liverpool', 'Stoke City', 'Swansea City',
       'Huddersfield Town', 'Tottenham Hotspur', 'Manchester City',
       'West Ham United', 'Fulham', 'Wolverhampton Wanderers',
       'Cardiff City', 'Sheffield United', 'Leeds United', 'Aston Villa',
       'Brentford', 'Norwich City'], dtype=object)

In [41]:
teamdf.sort_values(by='team')

,team,week,season,Home,Away,GF,GA,W,D,L,...,Performance_Int_sum,Performance_PK_sum,Performance_PKatt_sum,Performance_Press_sum,Performance_Sh_sum,Performance_SoT_sum,Performance_Tkl_sum,Performance_Touches_sum,SCA_GCA_sum,SCA_SCA_sum
0,Arsenal,1.0,1718,1,0,4,3,1,0,0,...,20.0,0.0,0.0,92.0,27.0,10.0,22.0,808.0,7.0,50.0
97,Arsenal,25.0,1819,0,1,1,3,0,0,1,...,13.0,0.0,0.0,133.0,4.0,2.0,8.0,550.0,2.0,8.0
98,Arsenal,25.0,2021,1,0,0,1,0,0,1,...,5.0,0.0,0.0,117.0,6.0,1.0,8.0,568.0,0.0,8.0
99,Arsenal,25.0,2122,0,1,4,2,1,0,0,...,13.0,1.0,1.0,208.0,13.0,3.0,21.0,487.0,5.0,21.0
100,Arsenal,26.0,1718,1,0,5,1,1,0,0,...,12.0,0.0,0.0,104.0,16.0,7.0,21.0,709.0,8.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3291,Wolverhampton Wanderers,25.0,2122,0,1,2,0,1,0,0,...,36.0,0.0,0.0,173.0,12.0,7.0,25.0,572.0,2.0,16.0
3290,Wolverhampton Wanderers,25.0,2021,1,0,1,0,1,0,0,...,17.0,0.0,0.0,89.0,16.0,5.0,14.0,478.0,1.0,25.0
3289,Wolverhampton Wanderers,25.0,1819,0,1,3,1,1,0,0,...,14.0,1.0,1.0,146.0,8.0,3.0,18.0,466.0,5.0,13.0
3287,Wolverhampton Wanderers,24.0,2122,1,0,0,1,0,0,1,...,18.0,0.0,0.0,90.0,15.0,4.0,12.0,665.0,0.0,27.0


In [42]:
fullseason = teamdf.drop(['W','D','L','week'], axis=1).groupby(['team', 'season']).mean().add_suffix('_mean').reset_index()

In [43]:
fullseason_sum = teamdf.drop(['W','D','L', 'week'], axis=1).groupby(['team', 'season']).sum().add_suffix('_sum').reset_index()

In [44]:
fullseason = fullseason.merge(fullseason_sum, on =['team', 'season'])
fullseason

,team,season,Carries_Carries_mean_mean,Carries_Prog_mean_mean,Dribbles_Att_mean_mean,Dribbles_Succ_mean_mean,Expected_npxG_mean_mean,Expected_xA_mean_mean,Expected_xG_mean_mean,Passes_Att_mean_mean,...,Performance_Int_sum_sum,Performance_PK_sum_sum,Performance_PKatt_sum_sum,Performance_Press_sum_sum,Performance_Sh_sum_sum,Performance_SoT_sum_sum,Performance_Tkl_sum_sum,Performance_Touches_sum_sum,SCA_GCA_sum_sum,SCA_SCA_sum_sum
0,Arsenal,1718,37.717296,4.711131,1.249816,0.763447,0.123818,0.096150,0.131436,48.041721,...,484.0,4.0,5.0,5449.0,590.0,227.0,690.0,29512.0,119.0,937.0
1,Arsenal,1819,34.502169,3.968334,1.060440,0.621168,0.102921,0.084601,0.110440,41.742337,...,442.0,4.0,5.0,5456.0,465.0,159.0,634.0,26669.0,111.0,766.0
2,Arsenal,2021,32.490457,3.817573,1.072778,0.570262,0.090519,0.067543,0.099165,40.057577,...,337.0,6.0,6.0,4685.0,455.0,141.0,510.0,25217.0,95.0,723.0
3,Arsenal,2122,29.816855,3.558994,1.045522,0.560382,0.103641,0.068329,0.114732,37.121915,...,476.0,5.0,8.0,4772.0,580.0,185.0,573.0,23735.0,88.0,921.0
4,Aston Villa,1718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,West Ham United,2122,26.936892,2.832431,0.969839,0.464511,0.090109,0.068020,0.098755,35.243509,...,539.0,4.0,6.0,4917.0,446.0,143.0,557.0,22216.0,100.0,674.0
108,Wolverhampton Wanderers,1718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109,Wolverhampton Wanderers,1819,26.944766,3.131434,1.140544,0.633603,0.083950,0.062247,0.089589,34.298728,...,498.0,4.0,4.0,5755.0,471.0,140.0,749.0,22844.0,80.0,704.0
110,Wolverhampton Wanderers,2021,27.551754,3.420248,1.488972,0.946429,0.070328,0.048744,0.076257,35.589777,...,388.0,4.0,4.0,5195.0,462.0,149.0,617.0,22963.0,56.0,748.0


In [45]:
table = teamdf.groupby(['team', 'season'])[['W', 'D', 'L']].sum()
table['points'] = table['W']*3 + table['D']*1
table

W  D   L  points
team                    season                   
Arsenal                 1718    19  6  13      63
                        1819    21  7  10      70
                        2021    18  7  13      61
                        2122    22  3  13      69
Aston Villa             1718     0  0   0       0
...                             .. ..  ..     ...
West Ham United         2122    16  8  14      56
Wolverhampton Wanderers 1718     0  0   0       0
                        1819    16  9  13      57
                        2021    12  9  17      45
                        2122    15  6  17      51

[112 rows x 4 columns]

In [46]:
table = table.reset_index()

In [47]:
fullseason = fullseason.merge(table[['team', 'season', 'points']], on =['team', 'season'])

In [48]:
teamdf = teamdf.sort_values(by=['season', 'team', 'week'])

In [49]:
teamdf['GF'] = teamdf['GF'].astype(np.float)
teamdf['GA'] = teamdf['GA'].astype(np.float)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [50]:
teamdf['Win'] = teamdf.W.to_numpy()
teamdf['Loss'] = teamdf.L.to_numpy()
teamdf['Draw'] = teamdf.D.to_numpy()
teamdf['GD'] = teamdf.GF - teamdf.GA

In [51]:
teamdf[['Opponent', 'week', 'season']]

,Opponent,week,season
0,Leicester City,1.0,1718
4,Stoke City,2.0,1718
8,Liverpool,3.0,1718
12,Bournemouth,4.0,1718
16,Chelsea,5.0,1718
...,...,...,...
3327,Burnley,34.0,2122
3331,Brighton & Hove Albion,35.0,2122
3335,Chelsea,36.0,2122
3339,Norwich City,37.0,2122


In [52]:
tfi = teamdf.set_index(['team', 'week', 'season'])
tcols = ['Performance_Sh', 'Performance_Touches', 'Performance_Tkl', 'Performance_Int', 'Performance_Blocks', 'SCA_SCA', 'Passes_Att', 'Passes_Cmp', 'Passes_Prog']

In [53]:
opcols = ['GF', 'GA', 'W', 'D', 'L', 'game_id', 'Carries_Carries_sum', 'Carries_Prog_sum',
       'Dribbles_Att_sum', 'Dribbles_Succ_sum', 'Expected_npxG_sum',
       'Expected_xA_sum', 'Expected_xG_sum', 'Passes_Att_sum',
       'Passes_Cmp_sum', 'Passes_Cmp%_sum', 'Passes_Prog_sum',
       'Performance_Ast_sum', 'Performance_Blocks_sum', 'Performance_CrdR_sum',
       'Performance_CrdY_sum', 'Performance_Gls_sum', 'Performance_Int_sum',
       'Performance_PK_sum', 'Performance_PKatt_sum', 'Performance_Press_sum',
       'Performance_Sh_sum', 'Performance_SoT_sum', 'Performance_Tkl_sum',
       'Performance_Touches_sum', 'SCA_GCA_sum', 'SCA_SCA_sum', 'GD']

# opcols = tcols

In [54]:
opponents = pd.DataFrame(teamdf[['Opponent', 'week', 'season']], columns = ['Opponent', 'week', 'season'])
opponents[opcols] = 0
for index, row in opponents.iterrows():
    opp, week, season = row['Opponent'], row['week'], row['season']
    key = (opp, week, season)
    try:
        opponents.loc[index, opcols] = tfi.loc[key, opcols]
    except:
        continue

In [55]:
teamdf = pd.concat([teamdf, opponents.drop(['Opponent', 'season', 'week'], axis=1).add_suffix('_opp')], axis=1)

In [56]:
train_cols = ['Carries_Carries_sum', 'Carries_Prog_sum',
       'Dribbles_Att_sum', 'Dribbles_Succ_sum', 'Expected_npxG_sum',
       'Expected_xA_sum', 'Expected_xG_sum', 'Passes_Att_sum',
       'Passes_Cmp_sum', 'Passes_Cmp%_sum', 'Passes_Prog_sum',
       'Performance_Ast_sum', 'Performance_Blocks_sum', 'Performance_CrdR_sum',
       'Performance_CrdY_sum', 'Performance_Gls_sum', 'Performance_Int_sum',
       'Performance_PK_sum', 'Performance_PKatt_sum', 'Performance_Press_sum',
       'Performance_Sh_sum', 'Performance_SoT_sum', 'Performance_Tkl_sum',
       'Performance_Touches_sum', 'SCA_GCA_sum', 'SCA_SCA_sum']

# train_cols = tcols

In [57]:
train_oppcols = [col + '_opp' for col in train_cols]

In [58]:
train_cols = train_cols + train_oppcols

In [59]:
train_df = teamdf[~teamdf.GD.isna()].copy()
train_df

,team,week,season,Home,Away,GF,GA,W,D,L,...,Performance_PK_sum_opp,Performance_PKatt_sum_opp,Performance_Press_sum_opp,Performance_Sh_sum_opp,Performance_SoT_sum_opp,Performance_Tkl_sum_opp,Performance_Touches_sum_opp,SCA_GCA_sum_opp,SCA_SCA_sum_opp,GD_opp
0,Arsenal,1.0,1718,1,0,4.0,3.0,1,0,0,...,0,0,139,6,4,18,435,6,7,-1
4,Arsenal,2.0,1718,0,1,0.0,1.0,0,0,1,...,0,0,204,11,4,21,378,2,17,1
8,Arsenal,3.0,1718,0,1,0.0,4.0,0,0,1,...,0,0,240,18,9,22,667,6,27,4
12,Arsenal,4.0,1718,1,0,3.0,0.0,1,0,0,...,0,0,174,8,2,14,574,0,11,-3
16,Arsenal,5.0,1718,0,1,0.0,0.0,0,1,0,...,0,0,189,13,4,18,700,0,18,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3327,Wolverhampton Wanderers,34.0,2122,0,1,0.0,1.0,0,0,1,...,0,0,209,14,5,22,495,2,22,1
3331,Wolverhampton Wanderers,35.0,2122,1,0,0.0,3.0,0,0,1,...,1,2,159,15,8,22,620,4,28,3
3335,Wolverhampton Wanderers,36.0,2122,0,1,2.0,2.0,0,1,0,...,1,1,129,19,5,21,629,3,30,0
3339,Wolverhampton Wanderers,37.0,2122,1,0,1.0,1.0,0,1,0,...,0,0,180,11,2,24,465,2,19,0


In [60]:
tmp = []
for col in train_cols:
    for tcol in tcols:
        if tcol in col:
            tmp.append(col)

In [61]:
train_cols = ['Passes_Att_sum',
 'Passes_Cmp_sum',
 'Passes_Prog_sum',
 'Performance_Blocks_sum',
 'Performance_Int_sum',
 'Performance_Sh_sum',
 'Performance_Tkl_sum',
 'Performance_Touches_sum',
 'SCA_SCA_sum',
 'Passes_Att_sum_opp',
 'Passes_Cmp_sum_opp',
 'Passes_Prog_sum_opp',
 'Performance_Blocks_sum_opp',
 'Performance_Int_sum_opp',
 'Performance_Sh_sum_opp',
 'Performance_Tkl_sum_opp',
 'Performance_Touches_sum_opp',
 'SCA_SCA_sum_opp']

In [62]:
train_df.columns

Index(['team', 'week', 'season', 'Home', 'Away', 'GF', 'GA', 'W', 'D', 'L',
       ...
       'Performance_PK_sum_opp', 'Performance_PKatt_sum_opp',
       'Performance_Press_sum_opp', 'Performance_Sh_sum_opp',
       'Performance_SoT_sum_opp', 'Performance_Tkl_sum_opp',
       'Performance_Touches_sum_opp', 'SCA_GCA_sum_opp', 'SCA_SCA_sum_opp',
       'GD_opp'],
      dtype='object', length=101)

In [63]:
train_df[['team'] + train_cols]

,team,Passes_Att_sum,Passes_Cmp_sum,Passes_Prog_sum,Performance_Blocks_sum,Performance_Int_sum,Performance_Sh_sum,Performance_Tkl_sum,Performance_Touches_sum,SCA_SCA_sum,Passes_Att_sum_opp,Passes_Cmp_sum_opp,Passes_Prog_sum_opp,Performance_Blocks_sum_opp,Performance_Int_sum_opp,Performance_Sh_sum_opp,Performance_Tkl_sum_opp,Performance_Touches_sum_opp,SCA_SCA_sum_opp
0,Arsenal,675.0,564.0,48.0,10.0,20.0,27.0,22.0,808.0,50.0,299,181,12,20,19,6,18,435,7
4,Arsenal,779.0,680.0,74.0,6.0,12.0,19.0,15.0,888.0,27.0,243,161,17,20,9,11,21,378,17
8,Arsenal,557.0,461.0,44.0,11.0,17.0,8.0,12.0,691.0,11.0,542,437,31,12,25,18,22,667,27
12,Arsenal,628.0,500.0,32.0,10.0,18.0,17.0,20.0,742.0,28.0,459,335,20,13,27,8,14,574,11
16,Arsenal,534.0,428.0,31.0,25.0,17.0,11.0,18.0,682.0,17.0,575,442,38,18,16,13,18,700,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3327,Wolverhampton Wanderers,624.0,516.0,36.0,13.0,15.0,10.0,27.0,746.0,17.0,352,268,25,17,24,14,22,495,22
3331,Wolverhampton Wanderers,483.0,389.0,34.0,16.0,14.0,11.0,21.0,584.0,18.0,514,423,34,12,11,15,22,620,28
3335,Wolverhampton Wanderers,358.0,280.0,20.0,15.0,24.0,14.0,17.0,486.0,17.0,512,432,44,9,13,19,21,629,30
3339,Wolverhampton Wanderers,625.0,542.0,53.0,9.0,15.0,17.0,8.0,725.0,25.0,363,275,14,10,22,11,24,465,19


In [64]:
train_df = train_df[~train_df.GD.isna()]

In [65]:
train_df = train_df.iloc[1:]
train_df

,team,week,season,Home,Away,GF,GA,W,D,L,...,Performance_PK_sum_opp,Performance_PKatt_sum_opp,Performance_Press_sum_opp,Performance_Sh_sum_opp,Performance_SoT_sum_opp,Performance_Tkl_sum_opp,Performance_Touches_sum_opp,SCA_GCA_sum_opp,SCA_SCA_sum_opp,GD_opp
4,Arsenal,2.0,1718,0,1,0.0,1.0,0,0,1,...,0,0,204,11,4,21,378,2,17,1
8,Arsenal,3.0,1718,0,1,0.0,4.0,0,0,1,...,0,0,240,18,9,22,667,6,27,4
12,Arsenal,4.0,1718,1,0,3.0,0.0,1,0,0,...,0,0,174,8,2,14,574,0,11,-3
16,Arsenal,5.0,1718,0,1,0.0,0.0,0,1,0,...,0,0,189,13,4,18,700,0,18,0
20,Arsenal,6.0,1718,1,0,2.0,0.0,1,0,0,...,0,0,145,6,1,19,421,0,8,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3327,Wolverhampton Wanderers,34.0,2122,0,1,0.0,1.0,0,0,1,...,0,0,209,14,5,22,495,2,22,1
3331,Wolverhampton Wanderers,35.0,2122,1,0,0.0,3.0,0,0,1,...,1,2,159,15,8,22,620,4,28,3
3335,Wolverhampton Wanderers,36.0,2122,0,1,2.0,2.0,0,1,0,...,1,1,129,19,5,21,629,3,30,0
3339,Wolverhampton Wanderers,37.0,2122,1,0,1.0,1.0,0,1,0,...,0,0,180,11,2,24,465,2,19,0


# First just performance measures to see what is important

In [66]:
from sklearn.metrics import *
from sklearn.model_selection import *
import lightgbm as lgb

In [67]:
clf = lgb.LGBMClassifier(n_estimators = 500)

In [68]:
xt, xv, yt, yv = train_test_split(train_df[train_cols], train_df['W'], test_size = 0.2)

In [69]:
clf.fit(xt, yt)

LGBMClassifier(n_estimators=500)

In [70]:
train_df['preds'] = clf.predict_proba(train_df[train_cols])[:, 1]

In [71]:
pp = clf.predict_proba(xv)[:, 1]
roc_auc_score(yv, pp)

0.7129928315412187

In [72]:
accuracy_score(yv, clf.predict(xv))

0.6694078947368421

In [73]:
f1_score(yv, clf.predict(xv))

0.5503355704697988

In [74]:
train_df.preds.max()

0.9999936210881326

In [75]:
roc_auc_score(train_df.W.values, train_df.preds.values)

0.9770268827847817

In [76]:
train_df[train_df.GD.isna()]

,team,week,season,Home,Away,GF,GA,W,D,L,...,Performance_PKatt_sum_opp,Performance_Press_sum_opp,Performance_Sh_sum_opp,Performance_SoT_sum_opp,Performance_Tkl_sum_opp,Performance_Touches_sum_opp,SCA_GCA_sum_opp,SCA_SCA_sum_opp,GD_opp,preds


In [77]:
mean_squared_error(train_df['GD'], train_df.preds)

2.851432798668752

In [78]:
best = {
#     'lambda_l1': 10,
#  'lambda_l2': 0.0012,
#  'num_leaves': 4,
#  'feature_fraction': 0.92,
#  'bagging_fraction': 0.88,
#  'bagging_freq': 1,
 'n_estimators': 500,
#  'min_child_samples': 15
}

In [79]:
features = train_cols
# Create out of folds array
y = train_df['W']
oof_predictions = np.zeros(train_df.shape[0])
# Create a KFold object
kfold = GroupKFold(n_splits = 3)
# Iterate through each fold
for fold, (trn_ind, val_ind) in enumerate(kfold.split(train_df, groups = train_df['week'])):
    print(f'Training fold {fold + 1}')
    x_train, x_val = train_df[features].iloc[trn_ind], train_df[features].iloc[val_ind]
    y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
    # Root mean squared percentage error weights
    model = lgb.LGBMClassifier(**best, importance_type='split')
    model.fit(x_train, y_train)
    # Add predictions to the out of folds array
    oof_predictions[val_ind] = model.predict_proba(x_val[features])[:, 1]
    # Predict the test set
print(roc_auc_score(y, oof_predictions))

Training fold 1
Training fold 2
Training fold 3
0.7253037735522005


In [80]:
w = train_df['Win']

In [81]:
feature_imp = pd.DataFrame(sorted(zip(model.feature_importances_,train_cols)), columns=['Value','Feature'])

In [82]:
feature_imp

,Value,Feature
0,487,Performance_Sh_sum_opp
1,601,Performance_Sh_sum
2,665,Performance_Blocks_sum
3,714,Performance_Tkl_sum
4,724,Performance_Blocks_sum_opp
5,749,SCA_SCA_sum_opp
6,762,Performance_Tkl_sum_opp
7,779,Performance_Int_sum
8,811,Performance_Int_sum_opp
9,860,SCA_SCA_sum


In [83]:
clf.booster_.save_model('clf.txt')